# Deploy AI Foundry with Models

### Prerequisites
- [Python 3.11 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = 'ai-foundry'
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming convention
resource_group_location = "eastus" # all the resources will be deployed in this location

# https://learn.microsoft.com/en-us/azure/ai-foundry/model-inference/concepts/models
models_config = [{"name": "DeepSeek-R1", "publisher": "DeepSeek", "version": "1", "sku": "GlobalStandard", "capacity": 1},
                 {"name": "Phi-4", "publisher": "Microsoft", "version": "3", "sku": "GlobalStandard", "capacity": 1},
                 {"name": "gpt-4o", "publisher": "OpenAI", "version": "2024-11-20", "sku": "GlobalStandard", "capacity": 1}
                 ]

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 11:54:34.597076 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [19]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 11:54:38.337576 :1s]
👉🏽 Current user: pablocastao@microsoft.com
👉🏽 Tenant ID: 16b3c013-d300-468d-ac64-7eda0820b6d3
👉🏽 Subscription ID: 06d043e2-5a2e-46bf-bf48-fffee525f377


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [21]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "modelsConfig": { "value": models_config }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-ai-foundry-sena 
👉🏽 Using existing resource group 'lab-ai-foundry-sena'
⚙️ Running: az deployment group create --name ai-foundry-sena --resource-group lab-ai-foundry-sena --template-file main.bicep --parameters params.json 
✅ Deployment 'ai-foundry-sena' succeeded ⌚ 11:57:04.151189 :49s]


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    project = utils.get_deployment_output(output, 'projectConnectionString', 'Project Connection String')    
    # Write the project and outputs to an .env file
    with open('../.env', 'w') as env_file:
        env_file.write(f"PROJECT_CONNECTION_STRING='{project}'\n")

⚙️ Running: az deployment group show --name ai-foundry-sena -g lab-ai-foundry-sena 
✅ Retrieved deployment: ai-foundry-sena ⌚ 12:00:25.870086 :2s]
👉🏽 Project Connection String: eastus.api.azureml.ms;06d043e2-5a2e-46bf-bf48-fffee525f377;lab-ai-foundry-sena;project-aytvwwktokp6w
